<a href="https://colab.research.google.com/github/erchiw/fa21_cs291_pj/blob/main/optimal_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cupy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve 

In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-c39ccd33-537c-36c4-f0a8-1a734ee815f7)


In [ ]:
np.random.seed(31)

In [ ]:
def add_gauss_noise(sigma, dim1, dim2):
    gauss_noise = sigma*np.random.normal(size=(dim1, dim2))
    return gauss_noise

def softmax_activation(z):
    exponentials = np.exp(z)
    exponentials_row_sums = np.sum(exponentials, axis=1)
    return exponentials / exponentials_row_sums[:, None]

def cross_entropy_loss(y_one_hot, activations):
    loss = -np.sum(y_one_hot * np.log(activations)) 
    return loss

def logis_pred(X, w):
    ## output a proba vector
    pred = softmax_activation(np.matmul(X,w))
    return pred

def run_noisyGD(niter, learning_rate, clip_threshold, sigma, X_train, y_train, X_test, y_test, w, GS, clip_or_not, sample_step):
    # sigma is noise multiplier
    n_train_sample = X_train.shape[0]
    n_test_sample = X_test.shape[0]
    n_feature = w.shape[0]
    n_class = w.shape[1]
    label_train = np.argmax(y_train, axis=1)
    label_test = np.argmax(y_test, axis=1)
    loss_tr = []
    loss_te = []
    auc_tr = []
    auc_te = []
    acc_tr = []
    acc_te = []
    grad_record = []
    for i in range(niter):
        print('\r', i, end='')
        y_pred = softmax_activation(np.matmul(X_train, w))
        if clip_or_not == False:
            w_gradients = -np.matmul(X_train.T, y_train-y_pred)
        else:
            w_gradients = -np.einsum('ij,ik->ijk', X_train, y_train-y_pred)
            clip = np.minimum(1, clip_threshold/np.linalg.norm(w_gradients, axis=(1,2), ord='fro'))
            w_gradients = np.einsum('i,ijk->jk', clip, w_gradients)
        w -= learning_rate * (w_gradients + add_gauss_noise(GS*sigma, n_feature, n_class))
        
        # calculated loss
        if niter-i<=10:
          # prediction
          y_pred_prob_train = logis_pred(X_train, w)
          y_pred_prob_test = logis_pred(X_test, w)
          # calculate loss
          loss_tr.append(cross_entropy_loss(y_train, y_pred_prob_train)/n_train_sample)
          loss_te.append(cross_entropy_loss(y_test, y_pred_prob_test)/n_test_sample)
          # calculate auc
          auc_tr.append(roc_auc_score(y_train.get(), y_pred_prob_train.get(), multi_class='ovr'))
          auc_te.append(roc_auc_score(y_test.get(), y_pred_prob_test.get(), multi_class='ovr'))
          # calculate acc
          acc_tr.append(float(sum(label_train == np.argmax(y_pred_prob_train, axis=1))) / n_train_sample)
          acc_te.append(float(sum(label_test == np.argmax(y_pred_prob_test, axis=1))) / n_test_sample)
          # record norm of gradient
          grad_record.append(np.linalg.norm(w_gradients, ord='fro'))
    return w, loss_tr, loss_te, auc_tr, auc_te, acc_tr, acc_te, grad_record

In [ ]:
# load data
trainX = np.load('./train_X_1.npy').astype('float32')
trainY = np.load('./train_Y_label.npy')
trainY_enc = np.load('./train_Y_enc.npy')
testX = np.load('./test_X_1.npy').astype('float32')
testY = np.load('./test_Y_label.npy')
testY_enc = np.load('./test_Y_enc.npy')
# check norm (less than sqrt(2))
print(np.max(np.linalg.norm(trainX, axis=1, ord=2)),
      np.max(np.linalg.norm(testX, axis=1, ord=2)))

1.4142137 1.4142137


In [ ]:
n_train_sample = trainX.shape[0]
n_test_sample = testX.shape[0]
n_feature = trainX.shape[1]
n_class = trainY_enc.shape[1]

# privacy
eps_list = np.arange(0.2, 5.2, 0.2)
delta = 1e-6
clip_list = np.array([0.05, 0.1, 0.2, 0.5, 0.8])

# lr and iter
niter_30 = np.load('./niter_30.npy')

# partition

# rep
rep = 10

In [ ]:
import numpy
noise_multiplier = 30
sample_step=1
j=0
for eps in eps_list:
  niter = niter_30[j]
  j=j+1
  
  # sample_step for 30
  if niter <= 500:
    sample_step = 25
  elif niter > 500 and niter <= 1000:
    sample_step = 50
  elif niter > 1000 and niter <= 5000:
    sample_step = 100
  elif niter > 5000 and niter <= 10000:
    sample_step = 500
  elif niter > 10000 and niter <= 50000:
    sample_step = 1000
  elif niter > 50000:
    sample_step = 1000

  for clip in clip_list:
    name = str(eps)[0:3]+'_'+str(clip)[0:4]
    if clip == 0.8 or clip == 0.5:
      lr = 5e-4
    elif clip == 0.2 or clip == 0.1 or clip == 0.05:
      lr = 5e-4
    
    if niter <= 10:
      dim1 = niter.get()
    elif niter > 10:
      dim1 = 10  
    tr_l_a = np.zeros((rep,dim1))
    te_l_a = np.zeros((rep,dim1))
    tr_u_a = np.zeros((rep,dim1))
    te_u_a = np.zeros((rep,dim1))
    tr_c_a = np.zeros((rep,dim1))
    te_c_a = np.zeros((rep,dim1))
    tr_g_a = np.zeros((rep,dim1))

    for i in range(rep):
      w_ini = np.zeros((n_feature, n_class))
      w, tr_l, te_l, tr_u, te_u, tr_c, te_c, grad  = run_noisyGD(niter.get(), lr, clip, noise_multiplier, trainX, trainY_enc, testX, testY_enc, w_ini, clip, True, sample_step)
      tr_l_a[i] = np.array(tr_l)
      te_l_a[i] = np.array(te_l)
      tr_u_a[i] = np.array(tr_u)
      te_u_a[i] = np.array(te_u)
      tr_c_a[i] = np.array(tr_c)
      te_c_a[i] = np.array(te_c)
      tr_g_a[i] = np.array(grad)
    numpy.save('./result_30_n/'+name+'_tr_l.npy', numpy.array(tr_l_a.get()).astype('float'))
    numpy.save('./result_30_n/'+name+'_te_l.npy', numpy.array(te_l_a.get()).astype('float'))
    numpy.save('./result_30_n/'+name+'_tr_u.npy', numpy.array(tr_u_a.get()).astype('float'))
    numpy.save('./result_30_n/'+name+'_te_u.npy', numpy.array(te_u_a.get()).astype('float'))
    numpy.save('./result_30_n/'+name+'_tr_c.npy', numpy.array(tr_c_a.get()).astype('float'))
    numpy.save('./result_30_n/'+name+'_te_c.npy', numpy.array(te_c_a.get()).astype('float'))
    numpy.save('./result_30_n/'+name+'_tr_g.npy', numpy.array(tr_g_a.get()).astype('float'))

 936

In [ ]:
!zip -r result30_optimal.zip result_30_n/

  adding: result_30_n/ (stored 0%)
  adding: result_30_n/4.0_0.2_tr_c.npy (deflated 76%)
  adding: result_30_n/3.2_0.2_te_u.npy (deflated 20%)
  adding: result_30_n/3.8_0.8_te_c.npy (deflated 75%)
  adding: result_30_n/0.8_0.05_te_c.npy (deflated 91%)
  adding: result_30_n/0.2_0.8_te_l.npy (deflated 18%)
  adding: result_30_n/3.2_0.5_tr_c.npy (deflated 71%)
  adding: result_30_n/4.8_0.8_tr_g.npy (deflated 5%)
  adding: result_30_n/2.4_0.8_te_u.npy (deflated 15%)
  adding: result_30_n/1.6_0.2_tr_u.npy (deflated 20%)
  adding: result_30_n/4.4_0.8_te_l.npy (deflated 10%)
  adding: result_30_n/4.2_0.2_te_c.npy (deflated 83%)
  adding: result_30_n/3.8_0.2_te_l.npy (deflated 11%)
  adding: result_30_n/0.6_0.1_te_u.npy (deflated 9%)
  adding: result_30_n/2.0_0.1_te_u.npy (deflated 20%)
  adding: result_30_n/5.0_0.2_te_c.npy (deflated 83%)
  adding: result_30_n/0.8_0.8_tr_c.npy (deflated 83%)
  adding: result_30_n/2.4_0.1_te_u.npy (deflated 20%)
  adding: result_30_n/3.0_0.2_te_c.npy (deflated

In [ ]:
from google.colab import files
files.download('/content/result30_optimal.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>